In [44]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2019)

In [46]:
# trn = pd.read_csv('./dataset/train_ver2.csv')
trn = pd.read_csv('./dataset/train_delete_empty_del.csv')
tst = pd.read_csv('./dataset/test_ver2_del.csv')

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,8,10,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
# del trn['cod_prov'], trn['tipodom']
# del tst['cod_prov'], tst['tipodom']

# trn.to_csv('./dataset/train_delete_empty_del.csv', index=False)
# tst.to_csv('./dataset/test_ver2_del.csv', index=False)

In [48]:
# dead = trn['indfall'] == 'S'
# trn = trn[~dead]

In [49]:
# trn.to_csv('./dataset/train_delete_empty_del.csv', index=False)

# Preprocessing

In [50]:
print('before tranin set shape : ', trn.shape)
prods = trn.columns[24:].tolist()

trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# # if some customer buy anyting, remove
# no_product = trn[prods].sum(axis=1) == 0
# trn = trn[~no_product]
# print('after tranin set shape : ', trn.shape)

before tranin set shape :  (11066226, 46)


In [51]:
for col in trn.columns[24:]:
    tst[col] = 0

df = pd.concat([trn, tst], axis=0)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


In [52]:
features = []

# 범주형을 0, 1, 2와 같은 꼴로 변경시킴
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

In [53]:
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

In [54]:
df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

In [55]:
df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

In [56]:
df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [57]:
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

In [58]:
features

['ind_empleado',
 'pais_residencia',
 'sexo',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'nomprov',
 'segmento',
 'age',
 'antiguedad',
 'renta',
 'ind_nuevo',
 'indrel',
 'indrel_1mes',
 'ind_actividad_cliente']

In [59]:
trn.columns[:23]

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'nomprov',
       'ind_actividad_cliente', 'renta', 'segmento', 'ind_ahor_fin_ult1'],
      dtype='object')

# Feature Engineering

In [60]:
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

In [61]:
df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [62]:
# lag_features = features.copy()

In [63]:
df.fillna(-99, inplace=True)

In [44]:
# for i in trn.groupby(['ncodpers'])['sexo']:
#     sl = len(group['sexo'].unique())
#     if sl != 1:
#         print(i)
#         print(sl)
trn.groupby(['ncodpers'])['sexo'].describe()

,count,freq,top,unique
ncodpers,,,,
15889,17,17,V,1
15890,17,17,V,1
15891,1,1,H,1
15892,17,17,H,1
15893,17,17,V,1
15894,17,17,V,1
15895,17,17,H,1
15896,12,12,V,1
15897,17,17,V,1


In [64]:
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    
    return int_date

In [65]:
# df['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [66]:
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

In [67]:
print(len(df_lag.columns))
print(len(df.columns))

51
51


In [68]:
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')
del df, df_lag

In [69]:
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

In [70]:
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

# Seperate validation set

In [71]:
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

In [72]:
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    #Extract new purchase
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype = np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [73]:
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

# Model training

In [74]:
param = {
    'booster' : 'gbtree',
    'max_depth' : 8,
    'nthread' : 8,
    'num_class' : len(prods),
    'objective' : 'multi:softprob',
    'silent' : 1,
    'eval_metric' : 'mlogloss',
    'eta' : 0.1,
    'min_child_weight' : 10,
    'colsample_bytree' : 0.8,
    'colsample_bylevel' : 0.9,
    'seed': 2019,
}

In [75]:
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [76]:
X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [77]:
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

[0]	train-mlogloss:2.66292	eval-mlogloss:2.67198
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.41947	eval-mlogloss:2.43226
[2]	train-mlogloss:2.24455	eval-mlogloss:2.25875
[3]	train-mlogloss:2.11228	eval-mlogloss:2.12801
[4]	train-mlogloss:2.0014	eval-mlogloss:2.01823
[5]	train-mlogloss:1.91338	eval-mlogloss:1.93156
[6]	train-mlogloss:1.83563	eval-mlogloss:1.85491
[7]	train-mlogloss:1.76611	eval-mlogloss:1.78566
[8]	train-mlogloss:1.70491	eval-mlogloss:1.72491
[9]	train-mlogloss:1.65473	eval-mlogloss:1.67533
[10]	train-mlogloss:1.6066	eval-mlogloss:1.62739
[11]	train-mlogloss:1.5655	eval-mlogloss:1.58649
[12]	train-mlogloss:1.52853	eval-mlogloss:1.54982
[13]	train-mlogloss:1.49494	eval-mlogloss:1.51686
[14]	train-mlogloss:1.46337	eval-mlogloss:1.48535
[15]	train-mlogloss:1.43535	eval-mlogloss:1.45758
[16]	train-mlogloss:1.41014	eval-mlogloss:1.43251
[17]	train-m

[161]	train-mlogloss:1.00006	eval-mlogloss:1.08678
[162]	train-mlogloss:0.999429	eval-mlogloss:1.08679
[163]	train-mlogloss:0.998887	eval-mlogloss:1.08677
[164]	train-mlogloss:0.998346	eval-mlogloss:1.08673
[165]	train-mlogloss:0.99781	eval-mlogloss:1.0867
[166]	train-mlogloss:0.997108	eval-mlogloss:1.08671
[167]	train-mlogloss:0.996648	eval-mlogloss:1.08668
[168]	train-mlogloss:0.99608	eval-mlogloss:1.08669
[169]	train-mlogloss:0.995433	eval-mlogloss:1.08671
[170]	train-mlogloss:0.994798	eval-mlogloss:1.08664
[171]	train-mlogloss:0.994195	eval-mlogloss:1.08659
[172]	train-mlogloss:0.993533	eval-mlogloss:1.08658
[173]	train-mlogloss:0.992856	eval-mlogloss:1.08663
[174]	train-mlogloss:0.992266	eval-mlogloss:1.08662
[175]	train-mlogloss:0.991827	eval-mlogloss:1.08663
[176]	train-mlogloss:0.99135	eval-mlogloss:1.08667
[177]	train-mlogloss:0.990783	eval-mlogloss:1.08668
[178]	train-mlogloss:0.990165	eval-mlogloss:1.08656
[179]	train-mlogloss:0.989727	eval-mlogloss:1.08653
[180]	train-mlogl

# Save trained model

In [78]:
import pickle

pickle.dump(model, open("model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit
best_ntree_limit

183

# Validation

In [79]:
from mapk import *

In [80]:
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]
    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [81]:
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

In [82]:
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

0.0427229053992809


In [83]:
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [84]:
# dead_vld = vld['indfall'] == 'S'
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [85]:
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y, p, ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])

In [86]:
print(mapk(add_vld_list, result_vld, 7, 0.0))

0.036512075033758036


# Training Total Dataset

In [87]:
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [88]:
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
print(best_ntree_limit)
print(model.best_ntree_limit)
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

226
183
[0]	train-mlogloss:2.66395
[1]	train-mlogloss:2.42069
[2]	train-mlogloss:2.24519
[3]	train-mlogloss:2.11257
[4]	train-mlogloss:2.0014
[5]	train-mlogloss:1.91305
[6]	train-mlogloss:1.83521
[7]	train-mlogloss:1.7656
[8]	train-mlogloss:1.70444
[9]	train-mlogloss:1.65411
[10]	train-mlogloss:1.60595
[11]	train-mlogloss:1.56478
[12]	train-mlogloss:1.52775
[13]	train-mlogloss:1.49413
[14]	train-mlogloss:1.46249
[15]	train-mlogloss:1.43445
[16]	train-mlogloss:1.4093
[17]	train-mlogloss:1.38497
[18]	train-mlogloss:1.36309
[19]	train-mlogloss:1.3428
[20]	train-mlogloss:1.32489
[21]	train-mlogloss:1.30815
[22]	train-mlogloss:1.2926
[23]	train-mlogloss:1.27868
[24]	train-mlogloss:1.26527
[25]	train-mlogloss:1.25289
[26]	train-mlogloss:1.24134
[27]	train-mlogloss:1.23123
[28]	train-mlogloss:1.22088
[29]	train-mlogloss:1.2114
[30]	train-mlogloss:1.20293
[31]	train-mlogloss:1.19458
[32]	train-mlogloss:1.18694
[33]	train-mlogloss:1.17944
[34]	train-mlogloss:1.17256
[35]	train-mlogloss:1.16606


In [89]:
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 19116)
('age', 18070)
('antiguedad', 16904)
('age_prev', 12179)
('antiguedad_prev', 12152)
('fecha_alta_month', 11207)
('nomprov', 10988)
('fecha_alta_year', 9065)
('renta_prev', 7870)
('canal_entrada', 7364)
('nomprov_prev', 5923)
('canal_entrada_prev', 4332)
('fecha_alta_month_prev', 4071)
('ind_recibo_ult1_prev', 3248)
('sexo', 3169)
('ind_ecue_fin_ult1_prev', 3119)
('fecha_alta_year_prev', 2991)
('ind_cco_fin_ult1_prev', 2845)
('ind_cno_fin_ult1_prev', 2633)
('segmento', 2225)
('ind_tjcr_fin_ult1_prev', 2084)
('ind_reca_fin_ult1_prev', 2054)
('segmento_prev', 1899)
('ind_nom_pens_ult1_prev', 1606)
('ind_valo_fin_ult1_prev', 1567)
('tiprel_1mes', 1556)
('ind_nomina_ult1_prev', 1509)
('ind_dela_fin_ult1_prev', 1450)
('ind_ctop_fin_ult1_prev', 1444)
('ind_actividad_cliente', 1310)
('sexo_prev', 1249)
('tiprel_1mes_prev', 1132)
('ind_ctpp_fin_ult1_prev', 1118)
('ind_fond_fin_ult1_prev', 1016)
('ind_ctma_fin_ult1_prev', 868)
('ind_actividad_cliente_prev', 8

In [90]:
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [91]:
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


# Save Final model

In [50]:
pickle.dump(model, open("model/xgb.baseline_total.pkl", "wb"))

In [ ]:
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y, p, ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])
print(mapk(add_vld_list, result_vld, 7, 0.0))

# Create submit file

In [92]:
submit_file = open('model/xgb.baseline.2019-06-20_baseline2', 'w')
submit_file.write('ncodpers,added_products\n')
count = 0
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
submit_file.close()